In [39]:
!pip install duckdb uuid pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 133.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 116.4 MB/s eta 0:00:00


In [94]:
import pandas as pd
import mmap
import os
import duckdb
import uuid
import json
import glob

In [49]:
df = pd.read_csv('../build/extracted_npi_offsets.csv', names=['NPI', 'offset', 'size'])

In [114]:
df

,NPI,offset,size
0,1962697342,99985389,4510
1,1992023816,99985389,4510
2,1861835290,99985389,4510
3,1801865787,99985389,4510
4,1861699548,99985389,4510
...,...,...,...
5424824,1154400513,84086296,599607
5424825,1174502934,59477315,1422148
5424826,1710414560,45976282,6567
5424827,1720224801,59477315,1422148


In [90]:
fbuff = open("../data/xxx/xaa", mode="r", encoding="utf8")

In [112]:
for row in df[df["NPI"] == 1316036643].to_records():
    fbuff.seek(row[2], 0)
    data = fbuff.read(row[3])
    obj = json.loads(data)
    pgs = []
    pg_tin = {}
    npi_tins = []
    negotiated_rates = []
    npi_negotiated_rates = []

    provider_groups = obj['provider_groups']
    #print(obj
    for pg in provider_groups:
        tin = pg['tin']['value']
        npis = pg['npi']

       # print(pg)
        if (pg_tin.get(tin) != True):
            pgs.append([tin, pg['tin']['type']])
            pg_tin[tin] = True

        for npi in npis:
            npi_tins.append([npi, tin])
    for nr in obj['negotiated_prices']:
        nr_id = str(uuid.uuid4())
        negotiated_rates.append([
            nr_id,
            tin,
            nr['negotiated_type'],
            nr['negotiated_rate'],
            nr['expiration_date'],
            nr['service_code'],
            nr['billing_class'],
            nr['billing_code_modifier'],
            nr['additional_information']
        ])
        for npi in npi_tins:
            npi_negotiated_rates.append([
                nr_id,
                npi[0]
            ])
    con.executemany('INSERT INTO provider_groups VALUES (?, ?)', pgs)
    con.executemany('INSERT INTO npi_tin VALUES (?, ?)', npi_tins)
    con.executemany('INSERT INTO negotiated_rates VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', negotiated_rates)

In [103]:
con.execute("select avg(negotiation_rate) from negotiated_rates").fetchall()

[(213.637,)]

In [92]:
con = duckdb.connect(database='1316036643_indexed.duckdb', read_only=False)

In [113]:
con.close()

In [110]:

con.execute("CREATE OR REPLACE TABLE extract(id UUID default uuid(), file VARCHAR, extracted_at TIMESTAMP, offs INTEGER, size INTEGER)")
con.execute("CREATE OR REPLACE TABLE provider_groups(tin VARCHAR, type VARCHAR)")
con.execute("CREATE OR REPLACE TABLE npi_tin(npi VARCHAR, tin VARCHAR)")
con.execute("CREATE OR REPLACE TABLE npi_extract(npi VARCHAR, extract UUID)")
con.execute("CREATE OR REPLACE TABLE negotiated_rates(id UUID default uuid(), tin VARCHAR, negotiated_type VARCHAR, negotiation_rate DOUBLE, expiration_date TIMESTAMP, service_code VARCHAR[], billing_class VARCHAR, billing_code_modifier VARCHAR[], additional_information VARCHAR)")
con.execute("CREATE OR REPLACE TABLE npi_negotiated_rates(negotiate_rate_id UUID, npi VARCHAR)")


In [111]:
con.execute("DELETE FROM provider_groups")
con.execute("DELETE FROM npi_tin")
con.execute("DELETE FROM negotiated_rates")
con.execute("DELETE FROM npi_negotiated_rates")

extracted/xaa/1.json file
extracted/xaa/10.json file
extracted/xaa/11.json file
extracted/xaa/2.json file
extracted/xaa/3.json file
extracted/xaa/4.json file
extracted/xaa/5.json file
extracted/xaa/6.json file
extracted/xaa/7.json file
extracted/xaa/8.json file
extracted/xaa/9.json file


In [32]:
con.executemany('INSERT INTO provider_groups VALUES (?, ?)', pgs)
con.executemany('INSERT INTO npi_tin VALUES (?, ?)', npi_tins)
con.executemany('INSERT INTO negotiated_rates VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', negotiated_rates)
con.executemany('INSERT INTO npi_negotiated_rates VALUES (?, ?)', npi_negotiated_rates)

In [104]:
!ls -larth indexed.duckdb

-rw-r--r-- 1 gitpod gitpod 8.6M Jul 12 13:58 indexed.duckdb


RuntimeError: Query interrupted